# Задачка про биткоины

Блокчейн биткоина хранит все когда-либо используемые адреса и транзакции между ними. Несмотря на кажущуюся анонимность данной криптовалюты, есть широкоизвестные методы, которые позволяют связать множество адресов, предполагая что они принадлежат одному и тому же владельцу (это может быть человек или компания). Так как все транзакции между адресами прозрачны, мы можем видеть сколько денег ушло от одного владельца к другому (это называется сash flow). 

***Необходимо посчитать сash flow между двумя кластерами адресов.***

***Важно***

Транзакция биткоина представляет собой набор входных и выходных адресов, а также указание сколько каждый адрес вносит биткоинов в данную транзакцию (для входных) или забирает из транзакции (для выходных адресов). Стоит заметить, что ничего не известно о том, сколько биткоинов переправляет конкретный входной адрес на любой другой из выходных. Также часть средств из суммы входов уходит в качестве сбора (fee) для майнеров при включении транзакции в новый блок. Мы можем найти fee (fee >= 0) как разницу между суммой входов и суммой выходов транзакции.

In [1]:
#импорт библиотек
import pandas as pd
import numpy as np

#### Таблица с адресами и кластерами

In [2]:
#посмотри таблицу с адресами и кластерами
address_clust = pd.read_csv('address_clust.csv')
display(address_clust.head())

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [3]:
address_clust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6996 entries, 0 to 6995
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   address_id  6996 non-null   int64
 1   cluster_id  6996 non-null   int64
dtypes: int64(2)
memory usage: 109.4 KB


In [4]:
#проверим на дубликаты
address_clust.duplicated().sum()

0

In [5]:
#проверим на пропущенные значения
address_clust.isnull().sum()

address_id    0
cluster_id    0
dtype: int64

#### Вывод

В таблице 6996 и 2 столбца:
- address_id - адрес куда/откуда отправлена транзакция
- cluster_id - кластер (1 или 2)

Нет ни дубликатов, ни пропущенных значений, типы данных в норме.

#### Таблица с транзакциями

In [6]:
#посмотри таблицу с транзакциями
address_stats = pd.read_csv('address_stats.csv')
display(address_stats.head())

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [7]:
address_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76724 entries, 0 to 76723
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              76724 non-null  int64  
 1   address_id      76724 non-null  int64  
 2   transaction_id  76724 non-null  int64  
 3   received        76724 non-null  float64
 4   sent            76724 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 2.9 MB


In [8]:
address_stats.duplicated().sum()

22279

In [10]:
#проверим на дубликаты
#address_stats.duplicated().sum()
address_stats = address_stats.drop_duplicates().reset_index(drop=True)
address_stats.duplicated().sum()

0

In [11]:
#проверим на пропущенные значения
address_stats.isnull().sum()

id                0
address_id        0
transaction_id    0
received          0
sent              0
dtype: int64

#### Вывод
В таблице 76724 строк и 5 колонок:

- id - просто id записи
- address_id - адрес куда/откуда отправлена транзакция
- transaction_id - транзакция
- received - сумма в сатоши (1 биткоин = 10^8 сатоши) полученная данным адресом в данной транзакции
- sent - сумма в сатоши отправленная данным адресом в данной транзакции


В таблице нет пропущенных значений, типы данных в норме. Можно было бы сменить тип данных на обычный int в столбцах с отправленными биткоинами, но, скорее всего, в реальности не будет ровных значений, поэтому оставим float.
В файле в столбце address_id есть дополнительные адреса, являющиеся внешними по отношению к кластеру 1 и 2, им мы сопоставляем кластер 0.

А дальше начинаются вопросы.

У нас было 22279 дубликатов. Это 29% от данных. С одной стороны это много, с другой, это полный дубль.
Я решила их удалить, потому что полный дубль увеличит нам суммы, а это нам не нужно.

Откуда взялись дубликаты? Для начала этот вопрос стоит адресовать тому, кто их собирал. Также, можно предположить, что это ошибка человеческого фактора: кто-то скопировал часть базы случайно. Может быть, если данные тоже где-то объединились, произошла ошибка там. 

В общем, цепочку надо раскручивать, а пока будем работать с тем, что есть.

#### Общая таблица

In [12]:
#объединим таблицы
new_clusters = address_stats.merge(address_clust, on = 'address_id', how = 'outer')

In [13]:
new_clusters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54445 entries, 0 to 54444
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              54445 non-null  int64  
 1   address_id      54445 non-null  int64  
 2   transaction_id  54445 non-null  int64  
 3   received        54445 non-null  float64
 4   sent            54445 non-null  float64
 5   cluster_id      37625 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 2.9 MB


In [14]:
#зададим нулевой кластер для оставшихся адресов
new_clusters['cluster_id'] = new_clusters['cluster_id'].fillna(value='0').astype(int)
display(new_clusters.head())

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,0
1,87154708,46402336,19166856,900000.0,0.0,0
2,87335816,46402336,19205422,0.0,900000.0,0
3,88124870,46402336,19374859,5500.0,0.0,0
4,95910795,46402336,21055878,0.0,5500.0,0


In [15]:
new_clusters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54445 entries, 0 to 54444
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              54445 non-null  int64  
 1   address_id      54445 non-null  int64  
 2   transaction_id  54445 non-null  int64  
 3   received        54445 non-null  float64
 4   sent            54445 non-null  float64
 5   cluster_id      54445 non-null  int32  
dtypes: float64(2), int32(1), int64(3)
memory usage: 2.7 MB


In [16]:
#странное
display(new_clusters.query('transaction_id==19162323'))

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,0
6,87134766,45919235,19162323,0.0,1071052.0,0
7,87134767,46529090,19162323,1000000.0,0.0,1
1701,87134768,46529091,19162323,1821052.0,0.0,0


#### Вывод
Итак, у нас получилась таблица с 54445 строками и 6 колонками:

    id - просто id записи
    address_id - адрес куда/откуда отправлена транзакция
    transaction_id - транзакция
    received - сумма в сатоши (1 биткоин = 10^8 сатоши) полученная данным адресом в данной транзакции
    sent - сумма в сатоши отправленная данным адресом в данной транзакции
    cluster_id - 0, 1 или 2 кластер

Дубликатов, пропущенных значений нет. Тип данных в cluster_id немного поправлен.

В ячейке выше привела пример странного. Как мы знаем, fee (fee >= 0) это разница между суммой входов и суммой выходов транзакции. Но тут не сходится. Биткоинов отправлено меньше, чем получено и fee будет отриательный. Может так и должно быть, но по идее это не верно и возникает вопрос, почему так вышло. Либо что-то не так с данными, либо это не очень честная транзакция.

#### Посчитаем биткоины

In [17]:
def func_sent_received(sent_cluster,received_cluster):
    """Функция для подсчета сколько биткоинов отправлено из одного кластера в другой.

    Функция принимает на вход 2 выборки с номерами кластера.
    """ 
    cluster_sent = sent_cluster
    cluster_received = received_cluster
    merge_clusters = cluster_received[['transaction_id','received']].merge(cluster_sent[['transaction_id','sent']])
    print(merge_clusters.sent.sum())

In [18]:
print('Биткоинов из кластера 1 в кластер 2 отправлено: ')
func_sent_received(new_clusters.query('cluster_id==1'),new_clusters.query('cluster_id==2'))

Биткоинов из кластера 1 в кластер 2 отправлено: 
8401735240.0


In [19]:
print('Биткоинов из кластера 2 в кластер 1 отправлено: ')
func_sent_received(new_clusters.query('cluster_id==2'),new_clusters.query('cluster_id==1'))

Биткоинов из кластера 2 в кластер 1 отправлено: 
51562110147.0


In [20]:
print('Биткоинов из кластера 1 в кластер 0 отправлено: ')
func_sent_received(new_clusters.query('cluster_id==1'),new_clusters.query('cluster_id==0'))

Биткоинов из кластера 1 в кластер 0 отправлено: 
55536195937.0


In [21]:
print('Биткоинов из кластера 0 в кластер 1 отправлено: ')
func_sent_received(new_clusters.query('cluster_id==0'),new_clusters.query('cluster_id==1'))

Биткоинов из кластера 0 в кластер 1 отправлено: 
102421428345.0


In [22]:
print('Биткоинов из кластера 2 в кластер 0 отправлено: ')
func_sent_received(new_clusters.query('cluster_id==2'),new_clusters.query('cluster_id==0'))

Биткоинов из кластера 2 в кластер 0 отправлено: 
302852000.0


In [23]:
print('Биткоинов из кластера 0 в кластер 2 отправлено: ')
func_sent_received(new_clusters.query('cluster_id==0'),new_clusters.query('cluster_id==2'))

Биткоинов из кластера 0 в кластер 2 отправлено: 
3065603309.0


Больше всего биткоинов отправлено ***из кластера 0 в кластер 1: 102 421 428 345.***

Далее, с небольшой разницей, транзакции биткоинов .***из кластера 1 в кластер 0(55 536 195 937)*** и ***из кластера 2 в кластер 1(51 562 110 147).***

В конце у нас транзакции ***из кластера 1 в кластер 2 (8 401 735 240)*** и ***из кластера 0 в кластер 2(3 065 603 309).***

В самом конце транзакции ***из кластера 2 в кластер 0 - 302 852 000 биткоинов.***


Всего биткоинов отправлено:

- из 0 кластера 105 487 031 654 биткоинов.

- из 1 кластера 63 937 931 177 биткоинов.

- из 2 кластера 51 864 962 147 биткоинов.


Общая сумма: 221 289 924 978 биткоинов.

Так как, нам не нужно рассчитывать, сколько биткоинов пришло, то и fee мы тут не учитываем.

#### Посчитаем fee

In [24]:
#выведем fee в отдельный столбец
fee = new_clusters.groupby(['transaction_id','cluster_id']).agg({'received':'sum', 'sent':'sum'}).reset_index()
fee['fee'] = fee['received']-fee['sent']
display(fee.head())

,transaction_id,cluster_id,received,sent,fee
0,19162323,0,1821052.0,2871052.0,-1050000.0
1,19162323,1,1000000.0,0.0,1000000.0
2,19166856,0,990000.0,0.0,990000.0
3,19166856,1,0.0,1000000.0,-1000000.0
4,19174421,0,4590000.0,4800000.0,-210000.0


In [25]:
print('Кластер 1 потратил на fee:', (fee.query("cluster_id==1 & fee >= 0")['fee']).sum(), 'биткоинов')
print('Кластер 2 потратил на fee:', (fee.query("cluster_id==2 & fee >= 0")['fee']).sum(), 'биткоинов')

Кластер 1 потратил на fee: 35670058860.0 биткоинов
Кластер 2 потратил на fee: 7071802861.0 биткоинов


#### Вывод

Здесь мы возвращаемся к замечанию выше про отрицательную fee. Повторюсь, что, возможно, так и должно быть. Если бы я работала в этой сфере,я бы значала точно. Но, по логике, комиссия не может быть отрицательной. 

В задании указано, что нужно брать значения fee от ноля, поэтому

***Кластер 2 тратит 35 670 058 860 биткоинов, а 1 кластер тратит 7 071 802 861 биткоинов.***